In [1]:
import pandas as pd
import numpy as np

def adjust_main(df):

    rows_to_drop = []
    # Players name standardization
    for i in df.index:
        if pd.isna(df.at[i,'Player']):
            rows_to_drop.append(i)  # avoiding empty rows
        else:
            df.at[i, 'Player'] = df.at[i, 'Player'].split('*')[0]   # removing the * mark

    # Some null fields should be 0.00 (e.g. 3pt shooting)
    df = df.fillna(0.00)

    # Dropping the duplicated rows
    df = df.drop(rows_to_drop)

    # Returning the standardized DataFrame
    return df  

def merging_df(df1,df2, keys):

    df1 = df1.join(df2.set_index(keys), on=keys)

#     df[['Status']] = df[['Status']].fillna(value='OOR')

    df1.to_csv(f'./data/{season}_std.csv',index=False)
    
    return df1
    
def adjust_mvp(df):
    
    # Players name standardization
    df['Status'] = 'Candidate'
    df.at[0,'Status'] = 'MVP'
    
    rows_to_drop = []
    flag = False
    
    for i in df.index:
        if pd.isna(df.at[i,'Player']):
            flag = True
        
        if flag is True:
            rows_to_drop.append(i)
    
    df = df.drop(rows_to_drop)

    df.to_csv(f'./data/mvp/{season}_mvp_std.csv', index=False)
     
    return df

def transfered_players(df):

    transfered_players = []
    rows_to_drop = []

    for i in df.index:
        if df.at[i,'Tm'] == 'TOT':
            transfered_players.append(df.at[i,'Player'])
            rows_to_drop.append(i)

    df_aux = df.drop(rows_to_drop)
    transfered_players_dict = {}

    for i in df_aux.index:
        if df_aux.at[i,'Player'] in transfered_players:
            pl = df_aux.at[i,'Player']
            if pl in transfered_players_dict.keys():
                transfered_players_dict[pl].extend([df_aux.at[i,'Tm'],int(df_aux.at[i,'G'])])
            else:
                transfered_players_dict[pl] = [df_aux.at[i,'Tm'],int(df_aux.at[i,'G'])]

    for player in transfered_players_dict.keys():
        max_played = 0
        for i in transfered_players_dict[player]:
            if isinstance(i,int):
                if i > max_played:
                    transfered_players_dict[player] = team
                    max_played = i
            else:
                team = i

    rows_to_drop.clear()

    for i in df.index:
        if df.at[i,'Player'] in transfered_players:
            if df.at[i,'Tm'] == 'TOT':
                df.at[i,'Tm'] = transfered_players_dict[df.at[i,'Player']]
            else:
                rows_to_drop.append(i)
                
    df = df.drop(rows_to_drop)

    return df

def teams_std(df, season):
    
    team_dict = {
    'Atlanta Hawks' : 'ATL',
    'Boston Celtics' : 'BOS',
    'Brooklyn Nets' : 'BRK',
    'Charlotte Hornets' : 'CHO', # 2015 - now
    'Chicago Bulls' : 'CHI',
    'Cleveland Cavaliers' : 'CLE',
    'Dallas Mavericks' : 'DAL',
    'Denver Nuggets' : 'DEN',
    'Detroit Pistons' : 'DET',
    'Golden State Warriors' : 'GSW',
    'Houston Rockets' : 'HOU',
    'Indiana Pacers' : 'IND',
    'Los Angeles Clippers' : 'LAC',
    'Los Angeles Lakers' : 'LAL',
    'Memphis Grizzlies' : 'MEM',
    'Miami Heat' : 'MIA',
    'Milwaukee Bucks' : 'MIL',
    'Minnesota Timberwolves' : 'MIN',
    'New Orleans Pelicans' : 'NOP',
    'New York Knicks' : 'NYK',
    'Oklahoma City Thunder' : 'OKC',
    'Orlando Magic' : 'ORL',
    'Philadelphia 76ers' : 'PHI',
    'Phoenix Suns' : 'PHO',
    'Portland Trail Blazers' : 'POR',
    'Sacramento Kings' : 'SAC',
    'San Antonio Spurs' : 'SAS',
    'Toronto Raptors' : 'TOR',
    'Utah Jazz' : 'UTA',
    'Washington Wizards' : 'WAS',
    'Seattle SuperSonics' : 'SEA',
    'San Diego Clippers' : 'SDC',
    'Kansas City Kings' : 'KCK',
    'Washington Bullets' : 'WSB',
    'New Jersey Nets' : 'NJN',
    'Charlotte Hornets CLASSIC' : 'CHH', # 1989 - 2002 
    'Vancouver Grizzlies' : 'VAN',
    'New Orleans Hornets' : 'NOH', # 2003 - 2005 / 2008 - 2014
    'Charlotte Bobcats' : 'CHA',
    'New Orleans/Oklahoma City Hornets' : 'NOK', # 2006 - 2007
    }

    chh_issue = np.arange(1989,2003,1)
    tm_list = []
    rows_to_drop = []

    for i in df.index:
        if df.at[i,'Team'] == 'Charlotte Hornets' and season in chh_issue:
            tm_list.append('CHH')
        elif df.at[i,'Team'] in team_dict.keys():
            tm_list.append(team_dict[df.at[i,'Team']])
        else:
            rows_to_drop.append(i)

    df = df.drop(rows_to_drop)        
    df['Tm'] = tm_list
    df = df[['Tm','W']]
    df.sort_values(r'W',ascending=False)
    
    return df

In [2]:
for season in np.arange(1981,2021,1):
    df1 = pd.read_csv(f'./basketball_reference_dbs/{season}_totals.csv')
    # Removing empty column
    df1 = df1.drop(columns=['GS'])
    df1 = adjust_main(df1)
    
    # Removing empty columns
    col = []
    for i in range(28):
        if i != 18 and i != 23:
            col.append(i)
    df2 = pd.read_csv(f'./basketball_reference_dbs/{season}_advanced.csv',usecols=col) 
    df2 = adjust_main(df2)
    
    # Merging both DataFrames
    df1 = merging_df(df1,df2,['Player','Pos', 'Age','G', 'MP','Tm']) 
    
    # Adjusting the MVP DataFrame
    col = ['Player', 'First','Share']
    df3 = pd.read_csv(f'./basketball_reference_dbs/mvp/{season}_mvp.csv',usecols=col)
    df3 = adjust_mvp(df3)
    
    # Merging again
    df1 = merging_df(df1,df3,['Player'])
    
    df1[['Status']] = df1[['Status']].fillna(value='OOR')
    df1 = df1.fillna(0.0)
    df1[['Season']] = season
    
    df1 = transfered_players(df1)
    
    df4 = pd.read_csv(f"./basketball_reference_dbs/teams/{season}_teams.csv")
    df4 = teams_std(df4,season)
    
    df1 = merging_df(df1,df4,['Tm'])
    
    data_types_dict = {'Age': 'int32', 'G': 'int32', 'MP': 'int32', 'FG': 'int32', 'FGA': 'int32', '3P': 'int32', '3PA': 'int32',
    '2P': 'int32', '2PA': 'int32', 'FT': 'int32', 'FTA': 'int32', 'ORB': 'int32', 'DRB': 'int32', 'TRB': 'int32', 'AST': 'int32', 
    'STL': 'int32', 'BLK': 'int32', 'TOV': 'int32', 'PF': 'int32', 'PTS': 'int32', 'First': 'int32', 'Season': 'object'}

    df1 = df1.astype(data_types_dict)
    
#     df1 = df1.dropna()
#     df1[r'W'] = df1[r'W'].astype('int32')
    
    df1.to_csv(f'./data/{season}_std.csv',index=False)

In [3]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 529 entries, 0 to 676
Data columns (total 53 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  529 non-null    object 
 1   Pos     529 non-null    object 
 2   Age     529 non-null    int32  
 3   Tm      529 non-null    object 
 4   G       529 non-null    int32  
 5   MP      529 non-null    int32  
 6   FG      529 non-null    int32  
 7   FGA     529 non-null    int32  
 8   FG%     529 non-null    float64
 9   3P      529 non-null    int32  
 10  3PA     529 non-null    int32  
 11  3P%     529 non-null    float64
 12  2P      529 non-null    int32  
 13  2PA     529 non-null    int32  
 14  2P%     529 non-null    float64
 15  eFG%    529 non-null    float64
 16  FT      529 non-null    int32  
 17  FTA     529 non-null    int32  
 18  FT%     529 non-null    float64
 19  ORB     529 non-null    int32  
 20  DRB     529 non-null    int32  
 21  TRB     529 non-null    int32  
 22  AS